In [1]:
from __future__ import annotations

from dataclasses import dataclass
import doctest
import re
import numpy as np


In [28]:
def extract_ints(s):
  """
  >>> extract_ints('')
  []
  >>> extract_ints('83 86  6 31 17  9 48 53')
  [83, 86, 6, 31, 17, 9, 48, 53]
  """
  return [int(n) for n in s.strip().split()]

@dataclass
class Card:
  num: int
  winning: list[int]
  yours: list[int]

  CARD_RE = re.compile(r'Card +(?P<num>\d+): +(?P<winning>(?:\d+ *)+) \| +(?P<yours>(?:\d+ *)+)')
  
  @classmethod
  def parse(cls, card_str: str) -> 'Card':
    """
    >>> Card.parse('Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53')
    Card(num=1, winning=[41, 48, 83, 86, 17], yours=[83, 86, 6, 31, 17, 9, 48, 53])
    """
    num, winning, yours = cls.CARD_RE.match(card_str).groups()
    num = int(num)
    winning = [int(c) for c in winning.strip().split()]
    yours = [int(c) for c in yours.strip().split()]
    return Card(num, winning, yours)
  
  def num_winning(self) -> int:
    return len(set(self.winning) & set(self.yours))

  def points(self) -> int:
    """
    >>> Card.parse('Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53').points()
    8
    >>> Card.parse('Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36').points()
    0
    """
    wins = self.num_winning()
    return 2 ** (wins - 1) if wins > 0 else 0

def solution_1(cards: list[Card]) -> int:
  return sum(c.points() for c in cards)

def solution_2(cards: list[Card]) -> int:
  num_cards = np.ones_like(cards)
  for c in cards:
    num_cards[c.num:c.num+c.num_winning()] += num_cards[c.num-1]
  return num_cards.sum()
  

In [29]:
test_case = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

test_cards = [Card.parse(l) for l in test_case.splitlines()]
print(solution_1(test_cards))
print(solution_2(test_cards))


13
30


In [30]:
doctest.testmod(verbose=False, report=True, exclude_empty=True)


TestResults(failed=0, attempted=5)

In [32]:
# Final answers
with open('../data/day04.txt') as f:
    cards = [Card.parse(l) for l in f.read().splitlines()]
    print('Part 1: ', solution_1(cards))
    print('Part 2: ', solution_2(cards))


Part 1:  24542
Part 2:  8736438
